In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.interpolate import barycentric_interpolate
from numpy import linalg as la
from scipy.optimize import root
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline

# Problem 1

In [ ]:
def cheb(N):
    x = np.cos((np.pi/N)*np.linspace(0,N,N+1))
    x.shape = (N+1,1)
    lin = np.linspace(0,N,N+1)
    lin.shape = (N+1,1)
    
    c = np.ones((N+1,1))
    c[0], c[-1] = 2., 2.
    c = c*(-1.)**lin
    X = x*np.ones(N+1) # broadcast along 2nd dimension (columns)
    
    dX = X - X.T
    
    D = (c*(1./c).T)/(dX + np.eye(N+1))
    D = D - np.diag(np.sum(D.T,axis=0))
    x.shape = (N+1,)
    # Here we return the differentiation matrix and the Chebyshev points,
    # numbered from x_0 = 1 to x_N = -1
    return D, x

In [ ]:
u = lambda x:np.exp(x)*np.cos(6*x)

actual_deriv = lambda x:-6*np.exp(x)*np.sin(6*x)+u(x)
x_even = np.linspace(-1,1,100)

for N in [6, 8, 10]:
    D, x = cheb(N)
    u_prime = np.dot(D,u(x))
    barycentric = barycentric_interpolate(x, u_prime, x_even)
    
    plt.plot(x, u_prime,'r',label='Chebyshev Approx: '+str(N))
    plt.plot(x_even, actual_deriv(x_even),'b',label='actual')
    plt.plot(x_even, barycentric,'k',label='Barycentric')
    plt.legend()
    plt.show()

# Problem 2

In [ ]:
f = lambda x:np.exp(2*x)

actual = lambda x:(-np.cosh(2)-np.sinh(2)*x+np.exp(2*x))/4

N=8
D, x = cheb(N)
DD = D.dot(D)
DD[0,:] = 0
DD[-1,:] = 0
DD[0,0] = 1
DD[-1,-1] = 1
F = f(x)
F[0] = 0
F[-1] = 0

U = la.solve(DD,F)

barycentric = barycentric_interpolate(x, U, np.linspace(-1,1,100))

lin = np.linspace(-1,1,100)
plt.plot(lin, barycentric,'r',label='Barycentric')
plt.plot(lin, actual(lin),'g',label='Actual')
plt.legend()
plt.show()

# Problem 3

In [ ]:
f = lambda x:np.exp(3*x)

N=8
D, x = cheb(N)
operator = D.dot(D)+D
operator[0,:] = 0
operator[-1,:] = 0
operator[0,0] = 1
operator[-1,-1] = 1
F = f(x)
F[0] = -1
F[-1] = 2

U = la.solve(operator,F)

barycentric = barycentric_interpolate(x, U, np.linspace(-1,1,100))

lin = np.linspace(-1,1,100)
plt.plot(lin, barycentric,'r',label='Barycentric')
plt.legend()
plt.show()

# Problem 4

In [ ]:
N = 20
D, x = cheb(N)
DD = D.dot(D)

def F(U):
    out = 4*np.dot(DD,U) - lam*np.sinh(lam*U)
    out[0] = U[0]-1
    out[-1] = U[-1]
    return out

for lam in [4, 8, 12]:
    guess = np.ones(N+1)
    sol = root(F,guess).x
    lin = np.linspace(-1,1,100)
    barycentric = barycentric_interpolate(x, sol, lin)
    plt.plot(.5*(lin+1), barycentric, label='lam= '+str(lam))
    plt.plot(.5*(x+1), sol)

plt.legend()
plt.show()
    

# Problem 5

In [ ]:
N = 50
D, x = cheb(N)
DD = np.dot(D,D)

def F(U):
    out = U*DD.dot(U)-D.dot(U)**2-1
    out[0] = U[0]-7
    out[-1] = U[-1]-1
    return out

guess = 2*np.ones(N+1)
sol = root(F,guess).x
lin = np.linspace(-1,1,100)
barycentric = barycentric_interpolate(x, sol, lin)

plt.plot(x, sol, label='actual')
plt.plot(lin, barycentric, label='barycentric')

theta = np.linspace(0,2*np.pi,401)
X, T = np.meshgrid(lin, theta)
Y, Z = barycentric*np.cos(T), barycentric*np.sin(T)
fig = plt.figure()
ax = fig.gca(projection="3d")
ax.plot_wireframe(X, Y, Z, rstride=10, cstride=10)
plt.show()

In [ ]:
print 2*np.pi*np.sum(sol*np.sqrt(1+np.dot(D,sol)**2))*2./50